# Logistic Regression
Logistic regression is a classification model which assumes that the probability of each class is proportional to a weighted sum of the input features.

In terms of mathematics, it is basically the same as linear regression except we apply a sigmoid activation to our output to map it between 0 and 1 as it represents a probability.

### $h = \sigma(XW)$

Even though Mean Squared Error Loss would work for this problem too, it has been shown that using Cross Entropy Loss leads to faster convergence.

### $ J = \sum_{i=1}^{m} - y^{(i)} \cdot \text{log}(h^{(i)}) + (1-y^{(i)}) \cdot \text{log}(1-h^{(i)})$

### Multi-class case
Consider the case where we are not performing binary classification but have multiple classes. How do we extend our regression model to work with this? We simply have multiple outputs where each output is a sigmoid applied to a linear combination of the inputs representing the probability of belonging to each class. This corresponds to adding an extra column in our weights vector so it is now a matrix. To calculate the cost we use the same cost function but sum the cost across all outputs.

To represent the labels, we can't use a single scalar value representing the class number anymore since our output is a vector so we wouldn't be able to compare them. Instead, we use one-hot encode the label. This is when we have a vector of length K, where K is the number of classes, which has a value of 0 for all the numbers except for the label class number which has a value of 1. If you think about it intuitively, this makes sense as we want our model to predict a probability of 1 for belonging to the label class and 0 for all other classes.


As usual, we use gradient descent to optimize this. Calculating our outputs and using that to calculate the gradient of the cost w.r.t each parameter so we can update the parameters to reduce the cost.